## Resources Link:

- Tutorial of [**Mixed-integer Programming: A Guide to Computational Decision-making**](https://www.toptal.com/algorithms/mixed-integer-programming) by Shanglun Wang
- S. Mitchell, [**An Introduction to pulp for Python Programmers**](https://github.com/coin-or/pulp/blob/master/doc/KPyCon2009/PulpForPythonProgrammers.pdf), 2009 

In [3]:
!pip install -U pulp

Here is an example in tutorial via PuLP (a popular operations research modeling library for Python). 
You can find information about [**PuLP here**](https://github.com/coin-or/pulp). 

## The simple example of coding

In [4]:
import pulp as pl

# declare some variables
# each variable is a binary variable that is either 0 or 1
# 1 means the item will go into the knapsack
a = pl.LpVariable("a", 0, 1, pl.LpInteger)
b = pl.LpVariable("b", 0, 1, pl.LpInteger)
c = pl.LpVariable("c", 0, 1, pl.LpInteger)
d = pl.LpVariable("d", 0, 1, pl.LpInteger)

# define the problem
prob = pl.LpProblem("knapsack", pl.LpMaximize)   ####################

# objective function - maximize value of objects in knapsack
prob += 5 * a + 7 * b + 2 * c + 10 * d                 #########################

# constraint - weight of objects cannot exceed 15
prob += 2 * a + 4 * b + 7 * c + 10 * d <= 15             #########################

status = prob.solve()  # solve using the default solver, which is cbc
print(pl.LpStatus[status])  # print the human-readable status

# print the values
print("a", pl.value(a))
print("b", pl.value(b))
print("c", pl.value(c))
print("d", pl.value(d))

Optimal
a 0.0
b 1.0
c 0.0
d 1.0


# But I think below one is not a good example to learn. 

As the management of the factory, we will want to **minimize labor costs**, but we want to **ensure sufficient coverage for every shift** to meet production demand.

Suppose we have five shifts with the following staffing demands:

Shift1 | Shift2 | Shift3 | Shift4 | Shift5 
-------|--------|--------|--------|-------
1 person|4 people|3 people|5 people|2 people

And, suppose we have the following workers:

Worker Name | Availability | Cost per Shift($)  
-------|--------|--------
Melisandre|1, 2, 5|20
Bran | 2,3,4,5 | 15 
Cersei|3,4|35
Daenerys|4,5|35
Theon |2,4,5|10 
Jon|1,3,5|25
Tyrion|2,4,5|30
Jaime|2,3,5|20
Arya|1,2,4|20

## Coding Our Solution

The coding of our solution is fairly straightforward. First, we will want to define our data:

In [9]:
import pulp as pl
import collections as cl

# data
shift_requirements = [1, 4, 3, 5, 2]
workers = {
    "Melisandre": {
        "availability": [0, 1, 4],
        "cost": 20
    },
    "Bran": {
        "availability": [1, 2, 3, 4],
        "cost": 15
    },
    "Cersei": {
        "availability": [2, 3],
        "cost": 35
    },
    "Daenerys": {
        "availability": [3, 4],
        "cost": 35
    },
    "Theon": {
        "availability": [1, 3, 4],
        "cost": 10
    },
    "Jon": {
        "availability": [0, 2, 4],
        "cost": 25
    },
    "Tyrion": {
        "availability": [1, 3, 4],
        "cost": 30
    },
    "Jaime": {
        "availability": [1, 2, 4],
        "cost": 20
    },
    "Arya": {
        "availability": [0, 1, 3],
        "cost": 20
    }
}

Next, we will want to define the model:

In [10]:
# define the model: we want to minimize the cost
prob = pl.LpProblem("scheduling", pl.LpMinimize)  ##################

# some model variables
cost = []
vars_by_shift = cl.defaultdict(list)

for worker, info in workers.items():
    for shift in info['availability']:
        worker_var = pl.LpVariable("%s_%s" % (worker, shift), 0, 1, pl.LpInteger)
        vars_by_shift[shift].append(worker_var)
        cost.append(worker_var * info['cost'])   

# set the objective to be the sum of cost
prob += sum(cost)                  ######################

# set the shift requirements
for shift, requirement in enumerate(shift_requirements):
    prob += sum(vars_by_shift[shift]) >= requirement               ######################

And now we just ask it to solve and print the results!

In [12]:
status = prob.solve()                ####################
print("Result:", pl.LpStatus[status])       #######################

results = []
for shift, vars in vars_by_shift.items():
    results.append({
        "shift": shift,
        "workers": [var.name for var in vars if var.varValue == 1],
    })

for result in sorted(results, key=lambda x: x['shift']):
    print("Shift:", result['shift'], 'workers:', ', '.join(result['workers']))


Result: Optimal
Shift: 0 workers: Melisandre_0
Shift: 1 workers: Bran_1, Theon_1, Jaime_1, Arya_1
Shift: 2 workers: Bran_2, Jon_2, Jaime_2
Shift: 3 workers: Bran_3, Cersei_3, Theon_3, Tyrion_3, Arya_3
Shift: 4 workers: Bran_4, Theon_4
